In [1]:
import torch
import torch.nn as nn
from torch import nn, optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
import pickle
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [2]:
# Load the data
file_path = "./static_data_summary.csv"
data = pd.read_csv(file_path)

# Extract the 'totpop' column
totpop = data['totpop'].values

# Convert 'totpop' to a tensor and move it to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
populations = torch.tensor(totpop, dtype=torch.float32).to(device)

#data.head()

In [3]:
# Extract features and target from the data
features = data[['SerHourBusRoutes', 'SerHourRailRoutes', 'BusStopDen', 'RailStationDen', 'popden', 'pctmale', 
                 'pctbachelor', 'young2', 'pcthisp', 'carown', 'pctlowinc', 'pctmidinc', 'pcthighinc', 
                 'pctsinfam2', 'CrimeDen', 'RdNetwkDen', 'InterstDen']].values
totpop = data['totpop'].values  #totpop is the target variable used for creating the monotonicity constraint
# Monotonicty Constraint: as totpop increases, travel demand should increase 
# Predict 'travel_demand', use random values for now
np.random.seed(0)
travel_demand = np.random.rand(len(features)) * 1000  # Replace with actual travel demand data if available

# Min-max normalization for features
features_min = np.min(features, axis=0)
features_max = np.max(features, axis=0)
features_normalized = (features - features_min) / (features_max - features_min)

# Min-max normalization for populations and travel_demand
pop_min = np.min(totpop)
pop_max = np.max(totpop)
populations_normalized = (totpop - pop_min) / (pop_max - pop_min)

travel_demand_min = np.min(travel_demand)
travel_demand_max = np.max(travel_demand)
travel_demand_normalized = (travel_demand - travel_demand_min) / (travel_demand_max - travel_demand_min)

# Convert to tensors
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_train = torch.tensor(features_normalized, dtype=torch.float32).to(device)
y_train = torch.tensor(travel_demand_normalized, dtype=torch.float32).to(device).view(-1, 1)
populations = torch.tensor(populations_normalized, dtype=torch.float32).to(device)

In [4]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [5]:
# Computes a penalty if the predictions do not uphold monotonic constraint 
def calculate_monotonicity_penalty(preds, populations, device):
    penalty = torch.zeros(1, device=device)
    for i in range(1, len(preds)):
        if populations[i] > populations[i - 1] and preds[i] < preds[i - 1]:
            penalty += torch.abs(preds[i] - preds[i - 1])
    return penalty

In [13]:
def calculate_loss(y_pred, label, criterion, lam, device, populations):
    cost1 = criterion(y_pred, label).double()
    monotonicity_penalty = calculate_monotonicity_penalty(y_pred, populations, device)
    cost = (1 - lam) * cost1 + lam * monotonicity_penalty
    return cost, cost1, monotonicity_penalty

In [14]:
# Define model, criterion, and optimizer
input_size = X_train.shape[1]
hidden_size = 50
output_size = 1
model = SimpleNN(input_size, hidden_size, output_size).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
lam = 0.5  # Lambda for the monotonicity penalty
# penalty_scaling_factor = 1e-9  # Scaling factor for the monotonicity penalty

# Training loop
# In each epoch, the model performs a forward pass, calculates the loss, computes gradients, and updates the parameters.
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    y_pred = model(X_train)
    
    # Calculate loss
    loss, cost1, monotonicity_penalty = calculate_loss(
        y_pred, y_train, criterion, lam, device, populations
    )
    
    # Backward pass and optimization
    loss.backward()
    optimizer.step()
    
    # Print loss for monitoring
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}, Cost1: {cost1.item()}, Monotonicity Penalty: {monotonicity_penalty.item()}')

print("Training completed")

Epoch [1/100], Loss: 3.019421339035034, Cost1: 0.45822200179100037, Monotonicity Penalty: 5.580620765686035
Epoch [2/100], Loss: 2.873983383178711, Cost1: 0.4535963535308838, Monotonicity Penalty: 5.294370651245117
Epoch [3/100], Loss: 2.734501838684082, Cost1: 0.44924360513687134, Monotonicity Penalty: 5.0197601318359375
Epoch [4/100], Loss: 2.599095582962036, Cost1: 0.44512513279914856, Monotonicity Penalty: 4.753066062927246
Epoch [5/100], Loss: 2.4674437046051025, Cost1: 0.44104406237602234, Monotonicity Penalty: 4.4938435554504395
Epoch [6/100], Loss: 2.343407154083252, Cost1: 0.4371221661567688, Monotonicity Penalty: 4.249691963195801
Epoch [7/100], Loss: 2.222346544265747, Cost1: 0.43339407444000244, Monotonicity Penalty: 4.011299133300781
Epoch [8/100], Loss: 2.1047897338867188, Cost1: 0.42983561754226685, Monotonicity Penalty: 3.7797436714172363
Epoch [9/100], Loss: 1.993485927581787, Cost1: 0.4264475703239441, Monotonicity Penalty: 3.5605242252349854
Epoch [10/100], Loss: 1.8